In [1]:
import os
import json
import csv
from IPython.display import display, Image
from datetime import datetime

In [ ]:
onlytweets_dir = '/home/zhaomin/Documents/saved_tweets/'
instances = []
num_context = 3
batch_filename = 'batch.csv'
url_prefix = "http://www.cse.unt.edu/~blanco/screenshot/"

column_names = ['instance_id', 'event', 'anchor_location', 'anchor_timestamp', 'anchor_url']
for i in range(10-num_context+1, 10+num_context+1):
    column_names.append(f"context{i}_url")
    column_names.append(f"context{i}_timestamp")

with open(batch_filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for event in os.listdir(onlytweets_dir):
        event_name = event.split("_")[0] + event.split("_")[1].split("-")[0]
        event_path = os.path.join(onlytweets_dir, event)
        days = os.listdir(event_path)
        for day in days:
            day_path = os.path.join(event_path, day)
            day_instances = os.listdir(day_path)
            for day_instance in day_instances:
                day_instance_path = os.path.join(day_path, day_instance)

                # load the anchor tweet
                anchor_filename = f"anchor_{day_instance}.json"
                anchor_filepath = os.path.join(day_instance_path, anchor_filename)
                with open(anchor_filepath, 'r') as anchor_jsonfile:
                    anchor_tweet = json.load(anchor_jsonfile)
            
                location_name = " ".join([token.capitalize() for token in anchor_tweet['location'].split(" ")])
                
                ele = {
                    'instance_id': f"{event_name}_{day_instance}",
                    'event': f"{event_name}",
                    'anchor_location': location_name,
                    'anchor_timestamp': anchor_tweet['created_at'],
                    'anchor_url': f"{url_prefix}{event_name}_{day_instance}_anchor_{day_instance}.png"
                }
                
                # load the 20 context tweets
                tweets = []
                filenames = os.listdir(day_instance_path)
                json_filenames = [x for x in filenames if x.endswith(".json") and not x.startswith("anchor")]
                for json_filename in json_filenames:
                    json_filepath = os.path.join(day_instance_path, json_filename)
                    with open(json_filepath, 'r') as jsonfile:
                        tweet = json.load(jsonfile)
                        tweets.append(tweet)
                context_tweets = sorted(tweets, key = lambda x: datetime.strptime(x['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))
                                
                # only keep part of the context tweets
                context_tweets = context_tweets[10-num_context:10+num_context]
                
                for index, context_tweet in enumerate(context_tweets):
                    context_id = context_tweet['id_str']
                    ele[f"context{index+(10-num_context+1)}_url"] = f"{url_prefix}{event_name}_{day_instance}_{context_id}.png"
                    context_timestamp = datetime.strptime(context_tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y').strftime('%I:%M %p - %b %d, %Y')
                    ele[f"context{index+(10-num_context+1)}_timestamp"] = context_timestamp

                writer.writerow(ele)
                
                all_tweets = context_tweets[:num_context] + [anchor_tweet] + context_tweets[num_context:]
                instances.append(all_tweets)
#                 break
#             break
#         break

In [51]:
event.split("_")[0] + event.split("_")[1].split("-")[0]

'memorialday2020'

In [50]:
anchor_tweet['created_at']

'Tue May 26 18:20:51 +0000 2020'

In [15]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a[4:16]

[5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6]

In [52]:
day_instance

'1265347181973835776'

In [19]:
from datetime import datetime

a = 'Mon Jun 10 02:56:52 +0000 2019'
# datetime.strptime(a, "%d-%b-%y")

# import time

datetime.strptime(a, '%a %b %d %H:%M:%S +0000 %Y')

datetime.datetime(2019, 6, 10, 2, 56, 52)

In [16]:
tweets[0]

{'created_at': 'Mon Jun 10 02:56:52 +0000 2019',
 'id': 1137916511123857408,
 'id_str': '1137916511123857408',
 'full_text': 'What does pollution, trash, and plastic all have in common? They all our a danger to our ocean.\xa0\nImagine a world where there were no plastic straws and plastic bags! Beautiful! 😍',
 'truncated': False,
 'display_text_range': [0, 178],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1136685248295706625,
  'id_str': '1136685248295706625',
  'name': 'Protect and Save our Oceans!',
  'screen_name': 'EndDirtyOceans',
  'location': 'Philadelphia, PA',
  'description': 'The ocean is being destroyed and we NEED to make a change NOW! Here to raise more awareness!🌊💪🐠',
  'ur

In [3]:
import csv

csv_filename = 'batch_3context.csv'
        
# generate the html file
html_filename = "display_context_anchor.html"
with open(html_filename, 'w') as htmlfile:

    with open(csv_filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # write the start
        htmlfile.write("<!DOCTYPE html>\n<html>\n<head>\n<title>Tweet Example</title>\n</head>\n<body>\n")

        # use a big table to display the tweet example
        htmlfile.write("""<table style=\"margin-left: auto; margin-right: auto; border: 1px solid black; line-height: 1.0em; width: 1200\">\n""")
        image_css = """style=\"display: block; margin-left: auto; margin-right: auto; max-width: 450px; max-height: 650px;\""""
        
        for row in reader:

            # 3 context tweets before
            htmlfile.write("<tr>\n")

            # context8_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context8_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context9_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context9_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context10_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context10_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # anchor tweet
            htmlfile.write("<tr>\n")

            # anchor_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['anchor_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # 3 context tweets after
            htmlfile.write("<tr>\n")

            # context11_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context11_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context12_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context12_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context13_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context13_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
        htmlfile.write("</table>\n")

        # write the end
        htmlfile.write("</body>\n</html>")    